<a href="https://colab.research.google.com/github/bemarina/text-clf-deep-learning/blob/main/pretrained-word-embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Example with pretrained word embeddings (Keras documentation)

###Import modules

In [1]:
import os

# Only the TensorFlow backend supports string inputs.
os.environ["KERAS_BACKEND"] = "tensorflow"

import pathlib
import numpy as np
import tensorflow.data as tf_data
import keras
from keras import layers

###Download and explore the data

In [2]:
#Download the Newsgroup20 data
data_path = keras.utils.get_file(
    "news20.tar.gz",
    "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",
    untar=True,
)

17329808/17329808 [==============================] - 5s 0us/step


In [3]:
data_dir = pathlib.Path(data_path).parent / "20_newsgroup"
dirnames = os.listdir(data_dir)
print("Number of directories:", len(dirnames))
print("Directory names:", dirnames)

fnames = os.listdir(data_dir / "comp.graphics")
print("Number of files in comp.graphics:", len(fnames))
print("Some example filenames:", fnames[:5])


Number of directories: 20
Directory names: ['comp.os.ms-windows.misc', 'rec.motorcycles', 'talk.religion.misc', 'sci.electronics', 'sci.med', 'sci.space', 'comp.sys.ibm.pc.hardware', 'rec.autos', 'sci.crypt', 'misc.forsale', 'talk.politics.mideast', 'rec.sport.baseball', 'comp.sys.mac.hardware', 'soc.religion.christian', 'talk.politics.misc', 'rec.sport.hockey', 'alt.atheism', 'comp.windows.x', 'comp.graphics', 'talk.politics.guns']
Number of files in comp.graphics: 1000
Some example filenames: ['38802', '38553', '38396', '38832', '38658']


In [4]:
data_dir

PosixPath('/root/.keras/datasets/20_newsgroup')

###Split data into training and validation sets

In [5]:
samples = []
labels = []
class_names = []
class_index = 0
for dirname in sorted(os.listdir(data_dir)):
    class_names.append(dirname)
    dirpath = data_dir / dirname
    fnames = os.listdir(dirpath)
    print("Processing %s, %d files found" % (dirname, len(fnames)))
    for fname in fnames:
        fpath = dirpath / fname
        f = open(fpath, encoding="latin-1")
        content = f.read()
        lines = content.split("\n")
        lines = lines[10:]
        content = "\n".join(lines)
        samples.append(content)
        labels.append(class_index)
    class_index += 1

print("Classes:", class_names)
print("Number of samples:", len(samples))

Processing alt.atheism, 1000 files found
Processing comp.graphics, 1000 files found
Processing comp.os.ms-windows.misc, 1000 files found
Processing comp.sys.ibm.pc.hardware, 1000 files found
Processing comp.sys.mac.hardware, 1000 files found
Processing comp.windows.x, 1000 files found
Processing misc.forsale, 1000 files found
Processing rec.autos, 1000 files found
Processing rec.motorcycles, 1000 files found
Processing rec.sport.baseball, 1000 files found
Processing rec.sport.hockey, 1000 files found
Processing sci.crypt, 1000 files found
Processing sci.electronics, 1000 files found
Processing sci.med, 1000 files found
Processing sci.space, 1000 files found
Processing soc.religion.christian, 997 files found
Processing talk.politics.guns, 1000 files found
Processing talk.politics.mideast, 1000 files found
Processing talk.politics.misc, 1000 files found
Processing talk.religion.misc, 1000 files found
Classes: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.ha

In [6]:
len(class_names)
# 20 classes
# almost 1000 files per class, so we can assume it's a balanced clf problem

20

In [7]:
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]


In [8]:
print(len(samples))
samples[0]
print(len(train_samples))
print(len(val_samples))

19997
15998
3999


###Vocabulary index

In [9]:
# Use only 20,000 words and truncate/pad as necessary
vectorizer = layers.TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf_data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

In [10]:
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'the', 'to', 'of']

In [11]:
# Vectorizing a test sentence
output = vectorizer([["the cat sat on the mat"]])
print(output.numpy()[0, :6])

voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

[   2 3822 1767   15    2 6293]


In [12]:
output = vectorizer([["the cat sat on the mat lashebet h h h h h h "]])
print(output.numpy())


word_index['h']
# 0 is for padding, 1 for words not in the vocabulary

[[   2 3822 1767   15    2 6293    1  864  864  864  864  864  864    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0   

864

In [13]:
word_index['the']

word_index['mat']
count=0
for k, v in word_index.items():
  print(k,v)
  count=count+1
  if count>10:
    break



 0
[UNK] 1
the 2
to 3
of 4
a 5
and 6
in 7
is 8
i 9
that 10


In [14]:
# Load the pretrained word embeddings
!wget https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
!unzip -q glove.6B.zip

--2024-11-04 22:34:27--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.02MB/s    in 2m 39s  

2024-11-04 22:37:06 (5.19 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [15]:
with open("glove.6B.100d.txt","r") as f:
    print(len(f.readlines()))  # This would give length of files.

400000


In [16]:
path_to_glove_file = "glove.6B.100d.txt"

with open(path_to_glove_file) as f:
  count=0
  for line in f:
    count+=1
    print(line)
    if count>1:
      break



the -0.038194 -0.24487 0.72812 -0.39961 0.083172 0.043953 -0.39141 0.3344 -0.57545 0.087459 0.28787 -0.06731 0.30906 -0.26384 -0.13231 -0.20757 0.33395 -0.33848 -0.31743 -0.48336 0.1464 -0.37304 0.34577 0.052041 0.44946 -0.46971 0.02628 -0.54155 -0.15518 -0.14107 -0.039722 0.28277 0.14393 0.23464 -0.31021 0.086173 0.20397 0.52624 0.17164 -0.082378 -0.71787 -0.41531 0.20335 -0.12763 0.41367 0.55187 0.57908 -0.33477 -0.36559 -0.54857 -0.062892 0.26584 0.30205 0.99775 -0.80481 -3.0243 0.01254 -0.36942 2.2167 0.72201 -0.24978 0.92136 0.034514 0.46745 1.1079 -0.19358 -0.074575 0.23353 -0.052062 -0.22044 0.057162 -0.15806 -0.30798 -0.41625 0.37972 0.15006 -0.53212 -0.2055 -1.2526 0.071624 0.70565 0.49744 -0.42063 0.26148 -1.538 -0.30223 -0.073438 -0.28312 0.37104 -0.25217 0.016215 -0.017099 -0.38984 0.87424 -0.72569 -0.51058 -0.52028 -0.1459 0.8278 0.27062

, -0.10767 0.11053 0.59812 -0.54361 0.67396 0.10663 0.038867 0.35481 0.06351 -0.094189 0.15786 -0.81665 0.14172 0.21939 0.58505 -0.52158

In [17]:
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [18]:
# vocabulary index: each word is mapped to an index
# if the word is not in the vocabulary, then the index is 1
# if the sentence is shorter and we need padding, then the index is 0
# to create a vector for a word: the columns are the words in the vocabulary
# the initial vector would be (theoretically)  0 0 0 1 0 0 0
# then these weights are updated during training to reflect similarities between
# words or a closer/farther relation between words

In [19]:
for k, v in embeddings_index.items():
  print(k, v)
  print(len(v))
  break

# each word is mapped into 100 dimensions

the [-0.038194 -0.24487   0.72812  -0.39961   0.083172  0.043953 -0.39141
  0.3344   -0.57545   0.087459  0.28787  -0.06731   0.30906  -0.26384
 -0.13231  -0.20757   0.33395  -0.33848  -0.31743  -0.48336   0.1464
 -0.37304   0.34577   0.052041  0.44946  -0.46971   0.02628  -0.54155
 -0.15518  -0.14107  -0.039722  0.28277   0.14393   0.23464  -0.31021
  0.086173  0.20397   0.52624   0.17164  -0.082378 -0.71787  -0.41531
  0.20335  -0.12763   0.41367   0.55187   0.57908  -0.33477  -0.36559
 -0.54857  -0.062892  0.26584   0.30205   0.99775  -0.80481  -3.0243
  0.01254  -0.36942   2.2167    0.72201  -0.24978   0.92136   0.034514
  0.46745   1.1079   -0.19358  -0.074575  0.23353  -0.052062 -0.22044
  0.057162 -0.15806  -0.30798  -0.41625   0.37972   0.15006  -0.53212
 -0.2055   -1.2526    0.071624  0.70565   0.49744  -0.42063   0.26148
 -1.538    -0.30223  -0.073438 -0.28312   0.37104  -0.25217   0.016215
 -0.017099 -0.38984   0.87424  -0.72569  -0.51058  -0.52028  -0.1459
  0.8278    0.270

In [20]:
# Prepare embedding matrix for the words we have in the vocabulary
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 17996 words (2004 misses)


In [21]:
len(voc)==17992+2008


True

In [22]:
# Build the Embedding layer using the pretrained embeddings (so we set
# trainable=False)
# Using the embedding_matrix built above
from keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    trainable=False,
)
embedding_layer.build((1,))
embedding_layer.set_weights([embedding_matrix])

In [23]:
# Build the model
# 1D convolution model with global max pooling and classifier at the end

int_sequences_input = keras.Input(shape=(None,), dtype="int32")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         2000200   
                                                                 
 conv1d (Conv1D)             (None, None, 128)         64128     
                                                                 
 max_pooling1d (MaxPooling1  (None, None, 128)         0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, None, 128)         82048     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, None, 128)         0         
 g1D)                                                        

In [24]:
# First, convert our list-of-strings data to NumPy arrays of integer indices.
# The arrays are right-padded.

# Train the model
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)



In [25]:
# We use categorical crossentropy as our loss since we're doing softmax
# classification. Moreover, we use sparse_categorical_crossentropy since our
# labels are integers.
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)
model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))

Epoch 1/20
125/125 [==============================] - 4s 28ms/step - loss: 2.6272 - acc: 0.1494 - val_loss: 2.0285 - val_acc: 0.2841
Epoch 2/20
125/125 [==============================] - 3s 25ms/step - loss: 1.9215 - acc: 0.3329 - val_loss: 1.6111 - val_acc: 0.4409
Epoch 3/20
125/125 [==============================] - 3s 25ms/step - loss: 1.4927 - acc: 0.4847 - val_loss: 1.3222 - val_acc: 0.5329
Epoch 4/20
125/125 [==============================] - 3s 25ms/step - loss: 1.2672 - acc: 0.5645 - val_loss: 1.1649 - val_acc: 0.6079
Epoch 5/20
125/125 [==============================] - 3s 25ms/step - loss: 1.0958 - acc: 0.6220 - val_loss: 1.1355 - val_acc: 0.6272
Epoch 6/20
125/125 [==============================] - 3s 25ms/step - loss: 0.9741 - acc: 0.6636 - val_loss: 1.0613 - val_acc: 0.6564
Epoch 7/20
125/125 [==============================] - 3s 25ms/step - loss: 0.8518 - acc: 0.7065 - val_loss: 1.0366 - val_acc: 0.6694
Epoch 8/20
125/125 [==============================] - 3s 25ms/step - 

In [29]:
import tensorflow as tf
tf.__version__

'2.15.0'

In [30]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = model(x)
end_to_end_model = keras.Model(string_input, preds)

probabilities = end_to_end_model(
    tf.convert_to_tensor(
        [["this message is about computer graphics and 3D modeling"]]
    )
)

print(class_names[np.argmax(probabilities[0])])


comp.graphics


In [32]:
input_data = tf.convert_to_tensor(
        [["there are a lot of electronic items in my house"]]
    )

predictions = end_to_end_model(input_data)


In [37]:
input_data = tf.convert_to_tensor(
        [["there are 3 phones, 3 laptops, 1 TV and 1 car in my house"]]
    )

predictions = end_to_end_model(input_data)


In [38]:
# 20 classes
np.argmax(predictions[0])
class_names[np.argmax(predictions[0])]

'misc.forsale'

In [40]:
# save the model in Google drive
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful